# Rosenbrocks system Analysis

All needed information for controller design of Rosenbrocks system

In [1]:
# Import the needed packages
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import control as cn
import modelreduction as mr

## System evaluation

Compute the poles and zeros of the tranfer function.

In [2]:
K = np.array([[10,1],[1,10]])
T = np.array([[10,10],[10,10]])
L = np.array([[5,3],[2,1]])
D = np.linalg.inv(K)
mr.AREA_BASED_APPROXIMATION(K,T,L,D)

(array([[  1.00000000e+00,   1.38777878e-17],
        [  0.00000000e+00,   1.00000000e+00]]),
 array([[ 12.02020202,  -0.2020202 ],
        [  0.1010101 ,   9.98989899]]),
 array([[3, 0],
        [0, 1]]))